**Few Shoot Prompting**

El Few-Shot Prompting aplicado a la gestión contextual de documentos representa una técnica avanzada de ingeniería de prompts donde se proporcionan ejemplos específicos y concisos para entrenar al modelo en la clasificación y selección del contexto más relevante. En lugar de procesar todos los documentos disponibles simultáneamente, este enfoque utiliza un conjunto reducido de ejemplos predefinidos que guían al modelo en la identificación del documento de referencia más apropiado para cada consulta. Esta metodología optimiza significativamente el consumo de tokens y mejora la precisión de las respuestas al establecer patrones claros de clasificación mediante ejemplos representativos, permitiendo una gestión más eficiente de los recursos computacionales y una mejor experiencia de usuario.

**Lab**

1. Import openAI dependences (load with conda install openai)

In [1]:
import openai
from typing import List, Dict, Tuple

2. Use the API Key from openAI page

In [2]:
#Add our API Key copied from openAI page
openai.api_key = "" #Example -> dw-bQyr9128F49VJRNKAnST32lbkFJ2Zv4qEhWo4R0Xh6yrh15

# Verificar que la API key está configurada
print(f"API Key configurada: {'*' * len(openai.api_key) if openai.api_key else 'No configurada'}")

API Key configurada: ***********************************************************************************************


3.1 Make a request 

In [12]:
import openai
from typing import List, Dict, Tuple

class ContextualAssistant:
    def __init__(self, api_key: str = None):
        if api_key:
            openai.api_key = api_key
            
        # Diccionario de documentos de referencia
        self.reference_docs = {
            "technical": """
            Los asistentes de IA consumen una cantidad significativa de tokens al procesar grandes 
            volúmenes de información contextual, lo que resulta en costos elevados y tiempos de 
            respuesta más largos. Cuando un usuario realiza consultas que requieren el conocimiento 
            de múltiples documentos de referencia, el sistema típicamente debe procesar todos estos 
            documentos simultáneamente, lo que resulta ineficiente tanto en términos de recursos 
            computacionales como económicos.
            """,
            
            "business": """
            Esta problemática es particularmente relevante en entornos empresariales, educativos
            y de servicio al cliente, donde se requiere acceso rápido y preciso a información
            específica contenida en diversos documentos. La falta de un sistema que gestione
            eficientemente el contexto documental resulta en mayor consumo de recursos computacionales,
            costos operativos elevados, tiempos de respuesta prolongados y menor precisión en las respuestas.
            """,

            "safety": """
            En caso de accidente laboral, siga inmediatamente este protocolo:
            1. Mantenga la calma y evalúe la situación. Si hay peligro inmediato, evacue el área.
            2. Para emergencias graves, llame inmediatamente al número de emergencias interno (ext. 555) o 
               al servicio médico de emergencia (911).
            3. No mueva a una persona herida a menos que esté en peligro inmediato.
            4. Localice el botiquín más cercano y proporcione primeros auxilios si está capacitado.
            5. Notifique a su supervisor inmediato y al departamento de Recursos Humanos dentro de las 
               primeras 2 horas del incidente.
            6. Complete el formulario de reporte de incidentes antes de finalizar su turno.
            7. Coopere con la investigación de seguridad posterior al incidente.
            """,

            "career": """
            El plan de desarrollo profesional en nuestra empresa se estructura en tres niveles:
            
            Nivel Junior (0-2 años):
            - Asignación de mentor senior
            - Participación en proyectos supervisados
            - Capacitación técnica mensual
            
            Nivel Intermedio (2-5 años):
            - Liderazgo de proyectos pequeños
            - Mentoría a nuevos empleados
            - Especializaciones técnicas
            
            Nivel Senior (5+ años):
            - Gestión de equipos y proyectos estratégicos
            - Participación en decisiones departamentales
            - Oportunidades de transferencia internacional
            
            Cada empleado tiene revisiones trimestrales para evaluar su progreso y ajustar su plan de desarrollo.
            """
        }
        
        # Ejemplos para clasificación
        self.classification_examples = [
            {
                "query": "¿Cómo afecta el consumo de tokens al rendimiento?",
                "context": "technical",
                "explanation": "Esta pregunta se refiere a aspectos técnicos del procesamiento."
            },
            {
                "query": "¿Qué impacto tiene en los costos operativos?",
                "context": "business",
                "explanation": "Esta pregunta se relaciona con aspectos empresariales y económicos."
            },
            {
                "query": "¿Qué debo hacer si alguien se lesiona en el trabajo?",
                "context": "safety",
                "explanation": "Esta pregunta se relaciona con protocolos de seguridad y accidentes."
            },
            {
                "query": "¿Cuáles son las oportunidades de crecimiento para un desarrollador junior?",
                "context": "career",
                "explanation": "Esta pregunta se relaciona con desarrollo profesional y carrera."
            }
        ]

    def classify_query(self, query: str) -> str:
        prompt = "Clasifica la siguiente consulta en una de estas categorías: 'technical', 'business', 'safety', o 'career'.\n\nEjemplos:\n"
        
        for example in self.classification_examples:
            prompt += f"\nConsulta: {example['query']}\n"
            prompt += f"Categoría: {example['context']}\n"
            prompt += f"Explicación: {example['explanation']}\n"
            
        prompt += f"\nNueva consulta: {query}\nCategoría:"
        
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=50,
            temperature=0.3
        )
        
        category = response.choices[0].message.content.strip().lower()
        
        if 'technical' in category:
            return 'technical'
        elif 'safety' in category:
            return 'safety'
        elif 'career' in category:
            return 'career'
        else:
            return 'business'

    def get_response(self, query: str) -> Tuple[str, str]:
        context_type = self.classify_query(query)
        selected_context = self.reference_docs[context_type]
        
        prompt = f"""Contexto: {selected_context}

Consulta: {query}

Genera una respuesta basada únicamente en la información proporcionada en el contexto anterior."""

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200,
            temperature=0.7
        )
        
        return response.choices[0].message.content.strip(), context_type

    def single_response(self, query: str):
        response, context_used = self.get_response(query)
        print(f"\nContexto utilizado: {context_used}")
        print(f"Respuesta: {response}")
        return response, context_used

# Pruebas
if __name__ == "__main__":
    assistant = ContextualAssistant()
    
    preguntas_prueba = [
        "¿Cuál es el impacto del procesamiento de documentos en los recursos?",
        "¿Qué debo hacer primero si veo un accidente?",
        "¿Cuánto tiempo toma llegar a nivel intermedio?",
        "¿Cómo afecta este sistema a los costos operativos?"
    ]
    
    for pregunta in preguntas_prueba:
        print("\n" + "="*50)
        print(f"Pregunta: {pregunta}")
        assistant.single_response(pregunta)


Pregunta: ¿Cuál es el impacto del procesamiento de documentos en los recursos?

Contexto utilizado: technical
Respuesta: El procesamiento de documentos consume una cantidad significativa de tokens, lo que resulta en costos elevados y tiempos de respuesta más largos, afectando los recursos computacionales y económicos.

Pregunta: ¿Qué debo hacer primero si veo un accidente?

Contexto utilizado: safety
Respuesta: Lo primero que debes hacer si ves un accidente es mantener la calma y evaluar la situación. Si hay peligro inmediato, evacua el área.

Pregunta: ¿Cuánto tiempo toma llegar a nivel intermedio?

Contexto utilizado: career
Respuesta: Toma de 2 a 5 años llegar al nivel intermedio en el plan de desarrollo profesional de la empresa.

Pregunta: ¿Cómo afecta este sistema a los costos operativos?

Contexto utilizado: business
Respuesta: La falta de un sistema que gestione eficientemente el contexto documental resulta en costos operativos elevados.
